# Chapter 7: Ensemble Learning and Random Forests

Asking a question to a crowd of thousands of people and aggregating the answer will likely give you a better answer than asking an expert. This is called *wisdom of the crowd*. 

Similarly, we can aggregate the predictions of a group of predictors (predictors in this case are models: classifiers or regressors) and we get better predictions with a group than with even the best individual predictor. 

A group of predictors is called an **Ensemble**, thus we have **Ensemble Learning**. An Ensemble Learning algorithm is called an **Ensemble method**.

For example, we can train a group of Decision Trees on different subsets of a training set, make predictions, then select the class which gets the most "votes" from the group of trees. An ensemble of Decision Trees is called a **Random Forest** and is one of the most powerful ML algorithms available; Ensemble methods frequently win ML competitions.

This chapter covers:
 - Bagging and Pasting
 - Boosting
 - Stacking
 - etc.

## Voting Classifiers
Suppose we have some different classification predictors (models), each one achieving around 80% accuracy: LRC, SVM, RFC (random forest), KNN, etc. 

A simple way to get an even better classifier is to aggregate the predictions of each classifier and predict the class that gets the most votes. A majority voting classifier is called a *Hard Voting* classifier.

The voting classifier will produce a higher accuracy than the individual classifiers, themselves. 

Even if our voting classifier is filled with *weak learners* (algorithms that are only slightly better than random guessing), the ensemble is usually still better than a strong learner, provided there are a lot of weak learners and they are diverse. *Ensemble methods work best when the predictors are as independent as possible from one another*.

Below see creation of a voting classifier:

```
# Imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 3 Different (and diverse) classifiers
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()


# Ensemble method: VotingClassifier
voting_clf = VotingClassifier(estimators=[('lr', log_clf), 
                                          ('rf', rnd_clf), ('svc', svm_clf)],
                              voting='hard')
voting_clf.fit(X_train, y_train)


from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
```

When running this code, the `voting_clf` should at least slightly outperform the individual classifiers.

If all classifiers are able to estimate class probabilites (have a predict_proba() method), then SciKit-Learn can predict the class with the highest probability, averaged over the individual classifiers. This is called **soft voting**. Soft voting **achieves higher accuracy than hard voting because it gives higher weight to highly confident votes**.

To *use soft voting*, replace `voting = 'hard'` with `voting = 'soft'`

Note: SVC does not output probabilities by default; it needs the hyperparameter `probability = True`

## Bagging and Pasting
With Voting Classifiers, we used diverse (very different) training algorithms (Support vector, Logistic regression, and random forest in the above code).

Another approach is to use many of the same training algorithm (predictor), but on different subsets of the training set. 

**Bagging**, short for *Bootstrap aggregating*, is taking subsets of the training set *with replacement*.

**Pasting** is taking subsets of the training set *without replacement*.

**Bagging and Pasting allow training instances to be sampled several times over multiple predictors, but ONLY BAGGING allows training instances to be samples several times for the same predictor, hence with replacement**.

Once all predictors are trained, the ensemble makes the prediction by aggregating the individual predictions. The aggregation function is usually the *statistical mode* (the most frequent prediction just like Hard Voting), or the average for a Regression task.

The individual predictors have a higher bias than if trained on the full training set, but the ensemble will have a similar bias and a lower variance.

Training and predictions in ensemble methods can be done in parallel by allocating CPU cores or using different servers; just another reason ensembles are so strong.

### Bagging and Pasting in Scikit-Learn
Bagging and Pasting are achieved by using the same class in SK-Learn.

```
# Imports
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Bagging/Pasting class
bag_clf = BaggingClassifier(
 DecisionTreeClassifier(), n_estimators=500,
 max_samples=100, bootstrap=True, n_jobs=-1)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
```

In the above code for the BaggingClassifier():
 - DecisionTreeClassifier() is the predictor
 - `n_estimators` is using the predictor (Decision Tree) 500 times
 - `max_samples = 100` means each of the 500 predictors will be trained on 100 training instances
 - `bootstrapping = True` means bagging is used
  - `= False` means pasting is used
 - `n_jobs = -1` means SK-Learn will use all CPU cores!
 
BaggingClassifier *automatically performs soft voting if the base classifier has a predict_proba() method*.

Note: Bagging generally performs better, but checking pasting with CV is a good idea sometimes.

### Out of Bag Evaluation
With bagging, since bootstrapping is used, some training instances are never seen by any classifier in the ensemble; these samples are said to be **out-of-bag (oob) instances**; because the *out-of-bag* instances were never seen, they can be used for evaluation on the predictor instead of a validation set.

With the BaggingClassifier() use the `oob_score = True` parameter to automatically evaluate on the out-of-bag instances after training. The oob_score is recoverable with the `.oob_score_` attribute.

```
 # Note the oob_score=True parameter
 bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                             n_estimators=500, bootstrap=True, 
                             n_jobs=-1, oob_score=True)

 bag_clf.fit(X_train, y_train)    # Fitting automatically evals the oob instances
 bag_clf.oob_score_               # Evaluation score on oob instances
```

We can also get the prediction probabilities with the `oob_decision_function_ ` attribute.

```
 bag_clf.oob_decision_function_
```

## Random Patches and Random Subspaces
BaggingClassifier also support sampling features as well as the instances.

Sampling the features is controlled by the two hyperparameter `max_features` and `bootstrap_features`, which work exactly the same way as sampling instances, just with the features instead. Thus, each predictor (model) is trained on a random subset of the input features.

This is particularly useful when dealing with high-dimensional inputs, like images.

Sampling both instances and features is called the **Random Patches method**.

Keeping all training instances (for all predictors in the ensemble? ; with `bootstrap = False` and `max_samples = 1.0`) but sampling features (`bootstrap_features = True` and/or `max_features` < 1.0) is called **Random Subspaces method**.

Note: using max_sample = 1.0 is saying to sample 100% of the instances for each predictor. Pretty sure.

Sampling features results in even more predictor diversity, trading a bit more bias for
a lower variance.

## Random Forests
Recall a Random Forest is an Ensemble of Decision Trees. Random Forests generally use bagging, but sometimes pasting. 

The RandomForestClassifier() (or RandomForestRegressor) is optimized for Decision Trees and better to use over the BaggingClassifier(). 

The following code trains 500 Decision trees with a maximum of 16 nodes per tree.

```
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
```

With a few exceptions, the RandomForestClassifier has all the hyperparameters of the DecisionTreeClassifier to control how the tree grows, and BaggingClassifier to control the ensemble itself.

The Random Forest, instead of searching for THE very best feature when splitting a node, it searches for the best feature in a random subset of nodes. This results in greater tree diversity, a higher bias, lower variance, and a generally better model.

The code below is "roughly equivalent to a RandomForestClassifier":

```
bag_clf = BaggingClassifier(
 DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
 n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)
```

### Extra Trees
Note, in the paragraph above that the Random Forest searches for the best feature in a random subset of features: this is done by searching for the best threshold for splitting. We can make the Random Forest even more random by making the threshold that it searches to split on, randomized. This is called an **Extremely Randomized Tree Ensemble** aka **"Extra Trees"**.

Once again, this increases bias, but reduces variance. **Training is much faster than random forests** because searching for the best threshold is one of the most time-consuming tasks; making the threshold random takes way less time. 

The `ExtraTreesClassifier` and `ExtraTreesRegressor` has identical API to the RandomForest equivalents and will use the same hyperparameters.

*It is difficult to know whether ExtraTrees or RandomForest performs better on a problem without using GridSeachCV.*

### Feature Importance
Another great quality of Random Forests is that SK-learn measures automatically measures a feature's importance by taking the average impurity reduction across all nodes that use that feature. It is a weighted average where the node's weight depends on the number of training samples associated with it.

The `feature_importances_` attribute of the Random Forest will sum to 1.

Below is an example on the iris dataset:

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.1034164636603887
sepal width (cm) 0.024645340382488886
petal length (cm) 0.4379600550708183
petal width (cm) 0.43397814088630415


Note that the petal length and width are seen as the most important features in predicting flowers.

Checking feature importances is also useful to see which pixels in a picture are the most important.

## Boosting
Boosting (originally Hypothesis Boosting) refers to any Ensemble method that combines several weak learners into a strong learner.

The general idea of most boosting methods is to train predictors (models) sequentially, each trying to correct its predecessor.

There are many boosting methods, but the two popular are: AdaBoost (Adaptive Boosting) and Gradient Boosting.

### Adaboost
Adaboosts correction technique is to have each new predictor (model algorithm) correct its predecessor by paying a bit more attention on the underfit training instances. The new predictors focus more and more on the hard cases.

Adaboosting algorithm:
1. Train a base classifier and make predictions on the training set. For all misclassified training instances, their weights are increased. 
2. A subsequent classifier trains on the training set with the updated instance weights; again predicting on the training set and increasing weights for all misclassified training instances
3. Repeat step 2 until going through all predictors.
4. Upon training all predictors, predictors with the highest accuracy have the most say, a higher weight, when predicting an instance.

A worthy note: The sequential architecture of some boosting techniques makes for a slow computation time because predictors cannot be trained in parallel, as $predictor_j$ depends on the updated weights of $predictor_i$, and $predictor_k$ depends on $predictor_j$ and $predictor_i$, and so on.

See pages 201-203 in Geron for the mathematics of updating weights.

SK-Learn uses a multiclass version of Adaboost called "SAMME", unless the predictors in the boosting sequence have a predict_proba() method and can output probabilities, then it uses "SAMME.R" which frequently performs better.

An example of AdaBoostClassifier():

```
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
 DecisionTreeClassifier(max_depth=1), n_estimators=200,
 algorithm="SAMME.R", learning_rate=0.5)
 
ada_clf.fit(X_train, y_train)
```

The above Adaboost algorithm's predictors are *Decision Stumps*.

*Decision Stumps* are decision trees of `max_depth = 1`, which only contain depth0 and depth 1, a root node and its two children.

If the Adaboost ensemble is overfitting, reduce the number of predictors with `n_estimators` or constrain (regularize) the base predictor itself.


### Gradient Boosting
The Gradient Boosting ensemble has the same sequential architecture as the Adaboost ensemble that looks to correct its predecessor, but instead of updating instance weights, the Gradient Boost ensemble fits on the residual errors (error for each instance) generated by the preceeding predictor.

Gradient Boosting works with Regression tasks, so we will boost DecisionTreeRegressors. This has a name in itself, Gradient Boosted Regression Trees.

The *following code exhibits Gradient Boosting without using the Gradient Boosting ensemble class*:

First create a Decision Tree Regressor
```
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)
```

Train another Decision Tree Regressor on the prediction errors of the first tree

```
y2 = y - tree_reg1.predict(X) # Errors of first tree
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2) # Fit with errors of first tree
```

Train a 3rd Decision Tree Regressor on the prediction errors of the second tree

```
y3 = y2 - tree_reg2.predict(X) # Errors of second tree
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3) # Fit with errors of second tree
```
and so on,

The above 3 code blocks is the ensemble, and it predicts by adding the predictions for all trees

```
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
```

Equivalently we can use the SK-Learn GBRT to achieve the same results:

```
# This does the exactly the same as previous 4 code blocks
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)
```

Similar to RandomForestRegressor, the GradientBoostingRegressor's hyperparameters control tree-growth and ensemble training.

"The learning_rate hyperparameter scales the contribution of each tree." A low learning rate means more trees in the ensemble, as each tree will have a smaller contribution, but how do we actually find the optimal number of trees in the Boosting sequence?

The Gradient Boost implementation has a staged_predict() method that **supports finding the optimal number of trees**.

The stage_predict() method evaluates the prediction error on a validation set at each iteration stage of training (train the first tree, eval on first; train the second tree; eval on first and second, and so on)(evaluates on 1 tree, 2 trees, 3 trees, etc). 

The following code finds the optimal number of trees:

```
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y)

# Create a GBRT with 120 trees
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

# Now, the MSE is evaluated for each set of 1,2,3,...,120 trees
# The 120th (119 in python) index of the errors list will have its prediction evaluated 
# by all 120 trees
# While the first (0th) index only evaluated on the first tree.

errors = [mean_squared_error(y_val, y_pred)
 for y_pred in gbrt.staged_predict(X_val)]
 
# Take the argmin is finding the index, also the number, of trees that gives the lowest MSE
bst_n_estimators = np.argmin(errors)

# recreate a GRBT with the best number of estimators, bst_n_estimators
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)
```

Or we can implement early stopping with the `warm_start=True` parameter to the GBRT class:

"The following code stops training when the validation error does not improve for five iterations in a row"

```
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
 gbrt.n_estimators = n_estimators
 gbrt.fit(X_train, y_train)
 y_pred = gbrt.predict(X_val)
 val_error = mean_squared_error(y_val, y_pred)
 if val_error < min_val_error:
 min_val_error = val_error
 error_going_up = 0
 else:
 error_going_up += 1
 if error_going_up == 5:
 break # early stopping
 
```

The GradientBoostRegressor also has a `subsample = ` hyperparameter that accepts 0 to 1.0 and is a fraction of training instances to be used for each predictor in the sequence. The training instances are selected randomly, and as usual, this increases the bias but decreases the variance in the ensemble. This is also called **Stochastic Gradient Boosting**.

Gradient boosting can also be done with different cost functions with the `loss` hyperparameter (see SK-Learn docs)

### XGBoost - Extreme Gradient Boosting
An optimized implementation of Gradient Boosting.

```
import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

xgb_reg.fit(X_train, y_train,
 eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)
```

"XGBoosts API is similar to SK-Learn's", and the book says to "check it out". (This is all that was given in the chapter)

## Stacking